## Data Cleaning using scikit learn 2

## 1. Missing Value Imputation
#### a) Numerical Variables
#### b) Categorical Variables
## 2. Different Strategy for Different Variables using
#### a) SimpleImputer
#### b) Pipeline
#### c) ColumnTransformer

## use for 3 methods
### 1. Fill missing value manually
### 2. Global Constant
### 3. Measure of central tendency (Mean, Median, Mode)

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
# load dataset
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print("Shape of train=",train.shape)
print("Shape of test=",test.shape)

Shape of train= (1460, 81)
Shape of test= (1459, 80)


In [4]:
x_train=train.drop(columns="SalePrice")
y_train=train["SalePrice"]
x_test=test.copy()
print("Shape of x_train=",x_train.shape)
print("Shape of y_train=",y_train.shape)
print("Shape of x_test=",x_test.shape)

Shape of x_train= (1460, 80)
Shape of y_train= (1460,)
Shape of x_test= (1459, 80)


# Missing value imputation

In [5]:
isnull_sum=x_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [7]:
# finding the numerical value which have missing value
num_vars=x_train.select_dtypes(include=['int64','float64']).columns
num_var_miss=[var for var in num_vars if isnull_sum[var]>0]

In [8]:
num_var_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [9]:
# finding the categorical value which have missing value
cat_vars=x_train.select_dtypes(include=['object']).columns
cat_var_miss=[var for var in cat_vars if isnull_sum[var]>0]
cat_var_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

### Pipeline- get sequence steps of Data Preprocessing

In [11]:
num_var_mean=['LotFrontage']
num_var_median=['MasVnrArea', 'GarageYrBlt']
cat_var_mode=['Alley','MasVnrType','BsmtQual', 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
              'Electrical','FireplaceQu']
cat_var_missing=['GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

In [13]:
num_var_mean_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="mean"))])
num_var_median_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median"))])
cat_var_mode_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])
cat_var_missing_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="constant",fill_value='Missing'))])

In [15]:
preprocessor=ColumnTransformer(transformers=[("mean_imputer",num_var_mean_imputer,num_var_mean),
                              ("median_imputer",num_var_median_imputer,num_var_median),
                               ("mode_imputer",cat_var_mode_imputer,cat_var_mode),
                              ("missing_imputer",cat_var_missing_imputer,cat_var_missing)])

In [16]:
preprocessor.fit(x_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('mean_imputer',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0))],
                                          verbose=False),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(memory=None,
                     

In [17]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('mean_imputer',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0))],
                                          verbose=False),
                                 ['LotFrontage']),
                                ('median_imputer',
                               

In [19]:
preprocessor.named_transformers_["mean_imputer"].named_steps["imputer"].statistics_

array([70.04995837])

In [20]:
train['LotFrontage'].mean()

70.04995836802665

In [21]:
preprocessor.named_transformers_["median_imputer"].named_steps["imputer"].statistics_

array([   0., 1980.])

In [22]:
preprocessor.named_transformers_["mode_imputer"].named_steps["imputer"].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [23]:
preprocessor.named_transformers_["missing_imputer"].named_steps["imputer"].statistics_

array(['Missing', 'Missing', 'Missing', 'Missing', 'Missing', 'Missing',
       'Missing'], dtype=object)

In [24]:
x_train_clean=preprocessor.transform(x_train)
x_test_clean=preprocessor.transform(x_test)

In [25]:
x_train_clean

array([[65.0, 196.0, 2003.0, ..., 'Missing', 'Missing', 'Missing'],
       [80.0, 0.0, 1976.0, ..., 'Missing', 'Missing', 'Missing'],
       [68.0, 162.0, 2001.0, ..., 'Missing', 'Missing', 'Missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'Missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'Missing', 'Missing', 'Missing'],
       [75.0, 0.0, 1965.0, ..., 'Missing', 'Missing', 'Missing']],
      dtype=object)

In [26]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='mean',
                                 verbose=0))],
           verbose=False),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='median',
                                 verbose=0))],
           verbose=False),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='most_frequent',
                                 verbose=0))],
           verbose=False),
  ['Alley',
   'MasVnrType',
   'BsmtQu

In [27]:
x_train_clean_miss_var=pd.DataFrame(x_train_clean,columns=num_var_mean+num_var_median+cat_var_mode+cat_var_missing)

In [28]:
x_train_clean_miss_var

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65,196,2003,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,Missing,Missing,Missing
1,80,0,1976,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
2,68,162,2001,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
3,60,0,1998,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,Missing,Missing,Missing
4,84,350,2000,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62,0,1999,Grvl,None,Gd,TA,No,Unf,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
1456,85,119,1978,Grvl,Stone,Gd,TA,No,ALQ,Rec,SBrkr,TA,Attchd,Unf,TA,TA,Missing,MnPrv,Missing
1457,66,0,1941,Grvl,None,TA,Gd,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,Missing,GdPrv,Shed
1458,68,0,1950,Grvl,None,TA,TA,Mn,GLQ,Rec,FuseA,Gd,Attchd,Unf,TA,TA,Missing,Missing,Missing


In [29]:
x_train_clean_miss_var.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65,196,2003,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,Missing,Missing,Missing
1,80,0,1976,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
2,68,162,2001,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing
3,60,0,1998,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,Missing,Missing,Missing
4,84,350,2000,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,Missing,Missing,Missing


In [30]:
x_train_clean_miss_var.isnull().sum().sum()

0

In [31]:
train["Alley"].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [32]:
x_train_clean_miss_var["Alley"].value_counts()

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [33]:
x_train_clean_miss_var["MiscFeature"].value_counts()

Missing    1406
Shed         49
Othr          2
Gar2          2
TenC          1
Name: MiscFeature, dtype: int64